In [89]:
from S15lib.instruments import TimeStampTDC1
import numpy as np

# Setup device, count singles and acquire timestamps

In [91]:
dev = TimeStampTDC1('/dev/tty.usbmodemTDC1_00141') # user your own device
dev.int_time = 2 # 2 seconds of integration time
print(dev.int_time)
print(dev.mode)
print(dev.get_singles())
times_ns, event_channels = dev.get_timestamps(1) # acquire timstamps for 1 s.


2.0
singles
[4, 0, 0, 0]
28 Bytes Recorded


# Arrival time correlations

In [4]:
info, t, histo = dev.count_g2(1)
print(info)
print(t, histo)

use python readevents
28 Bytes Recorded
{'channel1': 1, 'channel2': 0, 'total_time': 172042232}
[  0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34
  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70
  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100 102 104 106
 108 110 112 114 116 118 120 122 124 126 128 130 132 134 136 138 140 142
 144 146 148 150 152 154 156 158 160 162 164 166 168 170 172 174 176 178
 180 182 184 186 188 190 192 194 196 198 200 202 204 206 208 210 212 214
 216 218 220 222 224 226 228 230 232 234 236 238 240 242 244 246 248 250
 252 254 256 258 260 262 264 266 268 270 272 274 276 278 280 282 284 286
 288 290 292 294 296 298 300 302 304 306 308 310 312 314 316 318 320 322
 324 326 328 330 332 334 336 338 340 342 344 346 348 350 352 354 356 358
 360 362 364 366 368 370 372 374 376 378 380 382 384 386 388 390 392 394
 396 398 400 402 404 406 408 410 412 414 416 418 420 422 424 426 428 430
 432 434 436 438 440 442 444

# Get timestamps

In [92]:
acq_time = 1
cmd = '*RST;' + 'NIM;' + 'time ' + str(acq_time * 1000) + ';timestamp;counts?'
buffer = dev._com._stream_response_into_buffer(cmd, acq_time + 0.1)
bytes_hex = buffer[::-1].hex()

ts_word_list = [int(bytes_hex[i:i + 8], 16) for i in range(0, len(bytes_hex), 8)][::-1]

ts_list = []
pattern_list = []
periode_count = 0
periode_duration = 1<<27
prev_ts = -1
for ts_word in ts_word_list:
    time_stamp = (ts_word) >> 5
    pattern = ts_word & 0x1f
#     print(bin(ts_word), bin(pattern))
    print(time_stamp, bin(pattern))

    if prev_ts != -1 and time_stamp < prev_ts: #or ((prev_pattern & 0x10) == 0 and time_stamp == 67112824)):
            periode_count +=1
            print(periode_count)
    prev_ts = time_stamp
#     prev_pattern = pattern
    if pattern & 0x10 == 0:
        ts_list.append(time_stamp + periode_duration * periode_count)
        pattern_list.append(pattern)

# ts_list = list(map(lambda x: x*2, ts_list))
ts_list = np.array(ts_list) * 2
print(ts_list * 1e-9, pattern_list)

28 Bytes Recorded
67112824 0b10001
3960 0b10001
1
23482717 0b1
3960 0b10001
2
67112824 0b10001
3960 0b10001
3
5058517 0b1
[0.31540089 0.8154234 ] [1, 1]


In [94]:
dev._com.write(b'timestamp')

9

In [47]:
len('0b10000000000000011110111100010001')

34

In [77]:
format(((67112824 << 5) + 0x11), 'x')

'8001ef11'

In [66]:
48845117 < 67112824

True